# FFTW C2C Numba Sequential 576

- https://github.com/numba/numba/issues/5864
- https://numba.pydata.org/numba-doc/latest/user/withobjmode.html
- https://numba.pydata.org/numba-doc/dev/reference/types.html

In [41]:
#=--------------------------------------------------------------------=#

In [10]:
%%writefile bc2cs.py
import numpy as np, pyfftw as pf, time as tm
from numba import njit, objmode

t3 = tm.time()    # time measurement

@njit
def ff() :
    with objmode(t0 = 'f8') :
        t0 = tm.time()    # time measurement
    
    # data
    L = M = N = 576
    with objmode(u = 'complex128[:,:,:]') :  # annotate return type
        u = pf.empty_aligned( (N, M, L), dtype=np.complex128 )
    for k in range (u.shape[2]) :
        for j in range(u.shape[1]) :
            for i in range(u.shape[0]) :
                u[i, j, k] = complex( np.sin ( i + j + k + 3 ), 0 )
    
    with objmode(t1 = 'f8') :
        t1 = tm.time()    # time measurement
    
    # FFT
    with objmode(u = 'complex128[:,:,:]') :  # annotate return type
        u = pf.interfaces.numpy_fft.fftn(u)
    # checksum
    s = np.sum(u)
    
    with objmode(t2 = 'f8') :
        t2 = tm.time()    # time measurement
    
    return s, L, t0, t1, t2

# main
s, l, t0, t1, t2 = ff()

t4 = tm.time()    # time measurement

print(f"S:{s*1E-5:.0f}", end='')
print(f" | L:{l:0g}", end='')
print(f" | T1:{t1-t0:.4f}", end='')
print(f" | TF:{t2-t1:.4f}", end='')
print(f" | TT:{t2-t0:.4f}", end='')
print(f" | TO:{t4-t3:.4f}")

Overwriting bc2cs.py


In [11]:
! time python bc2cs.py

S:270-0j | L:576 | T1:23.3098 | TF:27.2651 | TT:50.5749 | TO:52.0067

real	0m54.142s
user	0m44.074s
sys	0m9.568s


In [18]:
! cp bc2cs.py /scratch${PWD#"/prj"}

In [19]:
%%writefile bc2cs.srm
#!/bin/bash
#SBATCH --job-name bc2cs       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:03:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
cd $dir/fft
                                              
# Executable
EXEC="python bc2cs.py"

# Start
echo '-- run --------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting bc2cs.srm


## Check

In [20]:
! sbatch  --partition=cpu_dev  bc2cs.srm

Submitted batch job 1333525


In [21]:
! squeue -n bc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1333525    cpu_dev   R  0:01     1   24


In [22]:
! squeue -n bc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [23]:
! cat /scratch${PWD#"/prj"}/slurm-1333525.out

- Job ID: 1333525
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1243
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python bc2cs.py
-- output -----------------------------
S:270-0j | L:576 | T1:21.6482 | TF:22.2247 | TT:43.8729 | TO:46.0559
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Run

In [24]:
! sbatch bc2cs.srm
! sbatch bc2cs.srm
! sbatch bc2cs.srm

Submitted batch job 1333528
Submitted batch job 1333529
Submitted batch job 1333530


In [25]:
! squeue -n bc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1333528  cpu_small  PD  0:00     1    1
           1333529  cpu_small  PD  0:00     1    1
           1333530  cpu_small  PD  0:00     1    1


In [1]:
! squeue -n bc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [2]:
! cat /scratch${PWD#"/prj"}/slurm-1333528.out
! cat /scratch${PWD#"/prj"}/slurm-1333529.out
! cat /scratch${PWD#"/prj"}/slurm-1333530.out

- Job ID: 1333528
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python bc2cs.py
-- output -----------------------------
S:270-0j | L:576 | T1:21.6802 | TF:22.2530 | TT:43.9332 | TO:54.3123
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1333529
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python bc2cs.py
-- output -----------------------------
S:270-0j | L:576 | T1:21.6738 | TF:22.2261 | TT:43.8999 | TO:45.8127
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1333530
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- Li